In [7]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
import db_multiproc_calc_neutral2 as dmc

rf_rate = 0.0475
k_do_download = False
all_sym = ['GCJ23', 'CLK23', 'NGJ23', 'ZCK23', 'ZWK23', 'ZSK23', 'E6M23', 'ZNM23',
           'SQJ23', 'NGK23', 'SIK23', 'HGK23']


# For a given option

cwd = os.getcwd()

executable_path = os.path.join(cwd,'chromedriver.exe')
display(executable_path)

date_str = datetime.now().strftime("%Y-%m-%d")
date_str = '2023-03-24'
file_date_str = date_str

download_path = os.path.join(cwd, 'local_download', file_date_str)
file_date_str

'C:\\Users\\mcbri\\PycharmProjects\\futureDataCapture\\chromedriver.exe'

'2023-03-24'

In [8]:
def get_put_call_ratio(df_iv):
    # This takes the IV datafrmae and calcs both the delta adjusted put call and
    # natural put call ratio from the barcharts data
    df_iv = df_iv.dropna().copy()
    df_iv = df_iv.loc[:, ['strike', 'call_put', 'open_interest', 'delta']]
    df_iv['Delta-Adj-Put-Call-Ratio'] = df_iv['open_interest'] * df_iv['delta']
    df_iv = df_iv.drop(columns=['delta'])
    df_iv = df_iv.set_index(['strike', 'call_put']).groupby('call_put').sum()
    df_iv = df_iv.rename(columns={'open_interest':'Put-Call-Ratio'})
    pc_temp = df_iv.T
    return (pc_temp.loc[:,'P'] / pc_temp.loc[:,'C'])

def get_iv_data_set(sym, file_date_str, barchart_datestr):
    # compile the implied vol dataframe from the files downloaded from barcharts
    df_exp_data = pd.read_csv(os.path.join('local_download',file_date_str, f'{sym}_{file_date_str}.txt'))
    exp_date_str = df_exp_data.values[0][0]
    exp_file_date_str = pd.to_datetime(exp_date_str).strftime('%m_%d_%y')

    iv_str = df_exp_data.values[0][1]
    all_iv = pd.to_numeric(iv_str.replace('%',''))/100

    px_data_path = os.path.join('local_download', file_date_str, f'{sym}_daily_historical-data-{barchart_datestr}.csv')
    opx_data_path = os.path.join('local_download', file_date_str, f'{sym}-options-american-options-exp-{exp_file_date_str}-show-all-%futuresoptionsview%-daily-{barchart_datestr}.csv')
    gr_data_path = os.path.join('local_download', file_date_str, f'{sym}-volatility-greeks-exp-{exp_file_date_str}-show-all-{barchart_datestr}.csv')

    # read the options price data
    fpx = pd.read_csv(px_data_path, parse_dates=['Time']).dropna()
    fpx['date'] = fpx['Time'].map(lambda x:pd.to_datetime(x))
    fpx = fpx.drop(columns=['Time'])
    fpx=fpx.set_index(['date']).sort_index()
    close_px = fpx.loc[barchart_datestr, :]['Last']

    # read in the option prices and transform
    opx = pd.read_csv(opx_data_path)
    opx = opx.iloc[:-1,:]
    opx['call_put']=opx['Strike'].map(lambda x:x[-1])
    opx['num_strike']=opx['Strike'].map(lambda x:pd.to_numeric(x[:-1].replace('-','.').replace(',','')))
    opx=opx.drop(labels=['Strike', 'Delta', 'Prev Open', 'Prev High', 'Prev Low', 'Prev Change', 'Premium'], axis=1)
    opx = opx.rename(columns={'num_strike': 'Strike',
                              'Prev Volume':'Volume',
                              'Open Interest':'open_interest',
                              'Last':'mean_price'})
    opx = opx.set_index(['Strike', 'call_put'])

    # read in the greeks and transform
    gr = pd.read_csv(gr_data_path)
    gr = gr.iloc[:-1,:]

    gr['symbol']=sym
    gr['option_expiration']=pd.to_datetime(exp_date_str)
    gr['Strike']=gr['Strike'].map(lambda x:pd.to_numeric(x))
    gr['IV']=pd.to_numeric(gr['IV'].str[:-1])/100
    gr['IV']=all_iv #RWM
    gr['Type'] = gr['Type'].map({'Put':'P', 'Call':'C'})
    gr=gr.drop(labels=['IV Skew', 'Time', 'Last'], axis=1)
    gr=gr.rename(columns={'Type':'call_put'})
    gr=gr.set_index(['Strike', 'call_put'])

    # concat and make columns lowercase
    t=pd.concat([opx, gr], axis=1)
    t['close_px'] = close_px
    t = t.reset_index()
    t.columns = map(str.lower, t.columns)

    return t

def clean_implied_vol(df_iv_file):
    x = df_iv_file.set_index(['strike', 'call_put']).sort_index().copy()
    y = x['iv'].unstack('call_put')
    idx_put_zero = y['P']==0
    idx_call_zero = y['C']==0

    idx_call_zero_replace = idx_call_zero * ~idx_put_zero
    idx_put_zero_replace = idx_put_zero * ~idx_call_zero
    y.loc[idx_call_zero_replace, 'C'] = y.loc[idx_call_zero_replace, 'P']
    y.loc[idx_put_zero_replace, 'P'] = y.loc[idx_put_zero_replace, 'C']
    y = y.rolling(3, center=True, min_periods=0).median()
    y = y.stack('call_put')
    y.name = 'clean_iv'
    x1 = pd.concat([x,y],axis=1)
    x1['iv'] = x1['clean_iv']
    x1.drop(columns='clean_iv', inplace=True)
    return x1

def generate_neutral_futures(sym, file_date_str, barchart_datestr):

    t = get_iv_data_set(sym, file_date_str, barchart_datestr)
    df_iv = clean_implied_vol(t).dropna().reset_index()

    query_date = barchart_datestr
    root_symbol = sym

    unique_strikes = sorted(t['strike'])
    spot_prices = np.array(dmc.calc_spot_price_levels(unique_strikes), dtype=float)

    agg_neutral, df_greeks_by_strike, exp_neutral = \
        dmc.calc_daily_neutral_values( \
            df_iv,
            query_date,
            rf_rate,
            root_symbol,
            spot_prices)

    agg_neutral['Close'] = t['close_px'].values[0]
    agg_neutral['Symbol'] = sym
    return [agg_neutral, df_greeks_by_strike, exp_neutral]

In [9]:
def bar_chart_get_driver(executable_path, download_path):
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    driver = webdriver.Chrome(executable_path=executable_path, options=chromeOptions)
    return driver

def bar_chart_login(driver):
    driver.get("https://www.barchart.com/login")
    driver.find_element(By.NAME, "email").click()
    driver.find_element(By.NAME, "email").send_keys("mcbride1689@gmail.com")
    driver.find_element(By.NAME, "password").click()
    driver.find_element(By.NAME, "password").send_keys("d5nJSz57A8C5")
    driver.find_element_by_class_name('login-button').click()
    return

def bar_chart_download(driver, date_str, sym, download=False):

    option_price_url = f"https://www.barchart.com/futures/quotes/{sym}/options?futuresOptionsTime=daily&moneyness=allRows"
    option_greek_url = f'https://www.barchart.com/futures/quotes/{sym}/volatility-greeks?moneyness=allRows'
    future_price_url = f'https://www.barchart.com/futures/quotes/{sym}/historical-download'
    driver.get(future_price_url)

    # download the price history
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, ".bc-price-history-checkboxes .checkbox").click()
    time.sleep(1)
    if download:
        driver.find_element(By.CSS_SELECTOR, ".add").click()
        time.sleep(2)

    # download the options prices
    driver.get(option_price_url)
    time.sleep(2)

    # get the expiration of the contract from the option price page
    temp_name = "#main-content-column > div > div:nth-child(4) > div > div:nth-child(1) > div > strong:nth-child(2)"
    expiration = driver.find_element(By.CSS_SELECTOR, temp_name).text
    temp_name = "#main-content-column > div > div:nth-child(4) > div > div.column.small-12.medium-4.text-medium-up-center > div > strong"
    iv_all = driver.find_element(By.CSS_SELECTOR, temp_name).text
    expiration = 'expiration,iv\n'+expiration+','+iv_all

    filename=os.path.join(download_path, f"{sym}_{date_str}.txt")
    with open(filename, "w") as text_file:
        print(expiration, file=text_file)

    time.sleep(2)
    if download:
        driver.find_element(By.CSS_SELECTOR, ".toolbar-button > span").click()
        time.sleep(2)

    # Get the option greeks
    time.sleep(2)
    driver.get(option_greek_url)
    time.sleep(3)
    if download:
        driver.find_element(By.CSS_SELECTOR, ".toolbar-button > span").click()
        time.sleep(3)
    return

In [10]:

if k_do_download:
    driver = bar_chart_get_driver(executable_path, download_path)
    bar_chart_login(driver)
    for sym in all_sym:
        bar_chart_download(driver, date_str, sym, k_do_download)
    driver.quit()

In [11]:
barchart_datestr = pd.to_datetime(file_date_str).strftime('%m-%d-%Y')

In [12]:
temp = []
for sym in all_sym:
    print(sym)
    temp.append(generate_neutral_futures(sym, file_date_str, barchart_datestr))

GCJ23
CLK23
NGJ23
ZCK23
ZWK23
ZSK23
E6M23
ZNM23
SQJ23
NGK23
SIK23
HGK23


In [13]:
# Pull the data together and get the delta put call ratio from the files
temp_iv_data = []
for sym in all_sym:
    df_iv = get_iv_data_set(sym, file_date_str, barchart_datestr)
    df_pcr = get_put_call_ratio(df_iv)
    df_iv['Put-Call-Ratio'] = df_pcr['Put-Call-Ratio']
    df_iv['Delta-Adj-Put-Call-Ratio'] = np.abs(df_pcr['Delta-Adj-Put-Call-Ratio'])
    df_iv = df_iv[['symbol', 'option_expiration', 'close_px', 'Put-Call-Ratio', 'Delta-Adj-Put-Call-Ratio']].tail(1)
    temp_iv_data.append(df_iv)
closepx_table = pd.concat(temp_iv_data).set_index('symbol')
closepx_table = closepx_table.rename(columns={'close_px':'Futures Close', 'option_expiration':'Opex Date'})

In [14]:
# Get the delta neutral and gamma neutral values and combine with final table
expn = [x[0] for x in temp]
final_output = pd.concat(expn).loc[-1,['symbol', 'delta_neutral', 'gamma_neutral' ]].set_index('symbol')
final_output = pd.concat([closepx_table, final_output], axis=1)
final_output = final_output.loc[:,['Opex Date', 'Futures Close', 'delta_neutral', 'gamma_neutral', 'Put-Call-Ratio', 'Delta-Adj-Put-Call-Ratio']]
final_output

,Opex Date,Futures Close,delta_neutral,gamma_neutral,Put-Call-Ratio,Delta-Adj-Put-Call-Ratio
symbol,,,,,,
GCJ23,2023-03-28,1983.800000,1883.741900,1873.337165,0.925211,0.069206
CLK23,2023-04-17,69.260000,71.301245,68.161003,0.673090,1.909350
NGJ23,2023-03-28,2.216000,2.533760,2.503956,0.552627,6.331629
ZCK23,2023-04-21,643.000000,639.986356,637.572442,1.035398,0.995602
ZWK23,2023-04-21,688.500000,703.639761,695.399850,0.554581,1.391668
ZSK23,2023-04-21,1428.250000,1506.037675,1532.234961,1.155915,5.316674
E6M23,2023-06-09,1.081400,1.036271,1.021188,0.474238,0.221210
ZNM23,2023-05-26,116.109375,111.276599,109.241391,0.667728,0.241212
SQJ23,2023-04-14,95.177500,95.931878,0.000000,1.325713,0.495771


In [15]:
final_output.to_csv(os.path.join(download_path,'finalout.csv'))

# Testing Follows

In [16]:
final_output_target = pd.read_csv(os.path.join(download_path,f'finalout.target.{file_date_str}.csv'), parse_dates=['Opex Date']).set_index('symbol')
test_target_sym = ['GCJ23', 'CLK23', 'NGJ23']

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mcbri\\PycharmProjects\\futureDataCapture\\local_download\\2023-03-24\\finalout.target.2023-03-24.csv'

In [ ]:
final_output.loc[test_target_sym,:]

In [ ]:
 final_output_target.loc[test_target_sym, :]

In [ ]:
# Check output against test
all_columns = final_output.columns

for c in all_columns:
    t1 = final_output.loc[test_target_sym,c]
    t2 = final_output_target.loc[test_target_sym,c]
    try:
        pd.testing.assert_series_equal(t1, t2, rtol=0.01, check_exact=False)
    except:
        display(pd.concat([t1, t2], axis=1))
